<a href="https://colab.research.google.com/github/kr7/udm/blob/main/udm_koszinusz_tavolsag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Koszinusz távolság számítása**

Importáljuk a szükséges szoftverkönyvtárakat:

In [1]:
import math
import numpy as np

from time import time

**1. megoldás: "középiskolai" szintű függvény**

Középiskolás koromban nagyjából egy ilyen függvényt implementáltam volna a koszinusz távolság számítására:

In [2]:
def cos_tavolsag(x, y):
  skalaris_szorzat=0
  for i in range(len(x)):
    skalaris_szorzat = skalaris_szorzat + x[i]*y[i]

  x_hossza = 0
  for i in range(len(x)):
    x_hossza = x_hossza + x[i]*x[i]
  x_hossza = math.sqrt(x_hossza)

  y_hossza = 0
  for i in range(len(y)):
    y_hossza = y_hossza + y[i]*y[i]
  y_hossza = math.sqrt(y_hossza)

  return 1 - skalaris_szorzat/(x_hossza*y_hossza)

In [3]:
cos_tavolsag( [2,4,5,1,6], [3,8,6,2,1])

0.21394263640508493

Megnézzük, hogy mennyire hatékony ez a függvény, ha "nagy tételben" akarunk koszinusz távolságokat számítani, ehhez 500x500 = 250.000 koszinusztávolságot foguk kiszámolni, és lemérjük a számoláshoz szükséges időt.

De miért is akarnánk "nagy tételben" számolni koszinusz távolságokat? Például egy nagy dokumentumhalmazban való hatékony keresés egyik alapja a koszniusz távolságok számítása, és ekkor SOK koszinusz távolságot kell számolnunk.

Lásd:

- https://en.wikipedia.org/wiki/Cosine_similarity ,
- https://medium.com/@arjunprakash027/understanding-cosine-similarity-a-key-concept-in-data-science-72a0fcc57599

In [4]:
data = np.random.rand(500,200)

In [5]:
t0 = time()
distances = np.zeros((500,500))
for i in range(500):
  for j in range(500):
    distances[i,j] = cos_tavolsag(data[i], data[j])
t1 = time()
print("Idő: ", t1-t0)

Idő:  60.34904146194458


**2. megoldás: gyorsítás Cython-nal**

Lásd: https://cython.org/

In [ ]:
%load_ext cython

In [8]:
%%cython

cimport cython
from libc.stdlib cimport malloc, free

def cos_tavolsag_cython(x, y):
  cdef float skalaris_szorzat=0
  cdef float x_hossza=0
  cdef float y_hossza=0
  cdef int i
  cdef float * x1
  cdef float * y1
  cdef int LEN_X
  cdef int LEN_Y

  LEN_X = len(x)
  LEN_Y = len(y)

  x1 = <float*>malloc(LEN_X*sizeof(float))
  y1 = <float*>malloc(LEN_Y*sizeof(float))

  for i in range(LEN_X):
    x1[i] = x[i]
  for i in range(LEN_Y):
    y1[i] = y[i]

  skalaris_szorzat=0
  for i in range(LEN_X):
    skalaris_szorzat = skalaris_szorzat + x1[i]*y1[i]

  x_hossza = 0
  for i in range(LEN_X):
    x_hossza = x_hossza + x1[i]*x1[i]
  x_hossza = x_hossza**0.5

  y_hossza = 0
  for i in range(LEN_Y):
    y_hossza = y_hossza + y1[i]*y1[i]
  y_hossza = y_hossza**0.5

  return 1 - skalaris_szorzat/(x_hossza*y_hossza)

In [9]:
t0 = time()
distances = np.zeros((500,500))
for i in range(500):
  for j in range(500):
    distances[i,j] = cos_tavolsag_cython(data[i], data[j])
t1 = time()
print("Idő: ", t1-t0)

Idő:  13.620433807373047


**3. megoldás: skaláris szorzat műveletét használva**

A @ operátorral (többek között) skaláris szorzatot számolhatunk hatékonyan. Megnézzük, ez vajon gyorsítja-e a koszinusz távolságok számítását:

In [10]:
def cos_tavolsag_kukaccal(x, y):

  skalaris_szorzat=x@y
  x_hossza = math.sqrt(x@x)
  y_hossza = math.sqrt(y@y)

  return 1 - skalaris_szorzat/(x_hossza*y_hossza)

In [11]:
t0 = time()
distances = np.zeros((500,500))
for i in range(500):
  for j in range(500):
    distances[i,j] = cos_tavolsag_kukaccal(data[i], data[j])
t1 = time()
print("Idő: ", t1-t0)

Idő:  1.3463561534881592


**4. megoldás: szoftverkönyvtárbeli függvény használata**

A sklearn szoftverkönyvtár tartalmaz egy függvényt a páronkénti koszinusztávolságok számításához. A következőkben kipróbáljuk ezt:

In [12]:
from sklearn.metrics.pairwise import cosine_distances

In [13]:
t0 = time()
distances = cosine_distances(data)
t1 = time()
print("Idő: ", t1-t0)

Idő:  0.013830900192260742
